In [73]:
# Importing modules
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import datetime as dt
from pyproj import CRS
import contextily as ctx
import requests
import geojson


In [74]:
# Reading the data into a dataframe
data = pd.read_json('https://iot.fvh.fi/opendata/uiras/uiras-meta.json', 
                    orient="index") 

# Creating shapely Point objects from the coordinates
data['geometry'] = data.apply(lambda row: 
                              Point(row['lon'], row['lat']), axis=1)

# Creating a geodataframe
geo = gpd.GeoDataFrame(data, 
                       geometry='geometry')

# Setting the CRS to EPSG:3857
geo = geo.set_crs(epsg=4326)
geo = geo.to_crs(epsg=3857)


In [75]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [76]:
# Dropping the Uunisaari data since there is no data
geo = geo.drop(index="003C62A8")


In [77]:
geo

,name,lat,lon,servicemap_url,site_url,fieldmap,geometry
70B3D57050001AB9,Pikkukosken uimaranta,60.227704,24.983821,https://palvelukartta.hel.fi/unit/41960,,NaN,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,60.207977,25.114849,https://palvelukartta.hel.fi/fi/unit/40157,,NaN,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,60.140588,24.915700,https://palvelukartta.hel.fi/fi/unit/45606,,NaN,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),60.171500,24.898300,https://palvelukartta.hel.fi/fi/unit/41717,http://www.tuk.fi,NaN,POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,60.175742,24.975318,https://palvelukartta.hel.fi/fi/unit/54929,https://www.sompasauna.fi,NaN,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,60.152320,25.015860,https://palvelukartta.hel.fi/fi/unit/50903,https://www.vasikkasaari.org,NaN,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),60.185540,25.040120,https://palvelukartta.hel.fi/fi/unit/41791,,NaN,POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),60.180100,25.068600,https://palvelukartta.hel.fi/fi/unit/57156,https://www.vartiosaari.fi,NaN,POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,60.198449,25.076416,https://palvelukartta.hel.fi/fi/unit/40386,,NaN,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),60.127797,24.691871,https://palvelukartta.hel.fi/fi/unit/39583,,NaN,POINT (2748686.506 8428245.610)


In [78]:
# Reading the temperature data
temps = pd.read_csv(
    "https://iot.fvh.fi/opendata/uiras/uiras-all-data.csv",
    parse_dates=['readable_time'],
    usecols=["readable_time", "temp_out1", "dev-id"]
)


In [79]:
temps

,readable_time,temp_out1,dev-id
0,2020-01-31 22:08:19.096000+00:00,1.62,70B3D57050001AB9
1,2020-01-31 22:18:20.076000+00:00,1.62,70B3D57050001AB9
2,2020-01-31 22:28:22.252000+00:00,1.56,70B3D57050001AB9
3,2020-01-31 22:38:22.101000+00:00,1.62,70B3D57050001AB9
4,2020-01-31 22:48:23.015000+00:00,1.56,70B3D57050001AB9
...,...,...,...
139768,2020-12-01 09:52:09.101000+00:00,2.50,70B3D57050001AB9
139769,2020-12-01 09:53:01.572000+00:00,5.25,70B3D57050004FE1
139770,2020-12-01 09:55:54.960000+00:00,4.75,70B3D57050001BBE
139771,2020-12-01 09:55:55.844000+00:00,4.50,70B3D57050004C07


In [80]:
# Reversing the dataframe
temps = temps[::-1]


In [81]:
temps.head(20)

,readable_time,temp_out1,dev-id
139772,2020-12-01 09:57:12.273000+00:00,4.56,70B3D57050004FE6
139771,2020-12-01 09:55:55.844000+00:00,4.50,70B3D57050004C07
139770,2020-12-01 09:55:54.960000+00:00,4.75,70B3D57050001BBE
139769,2020-12-01 09:53:01.572000+00:00,5.25,70B3D57050004FE1
139768,2020-12-01 09:52:09.101000+00:00,2.50,70B3D57050001AB9
139767,2020-12-01 09:51:18.013000+00:00,6.25,70B3D57050005037
139766,2020-12-01 09:45:55.084000+00:00,5.62,70B3D57050004FB9
139765,2020-12-01 09:44:58.703000+00:00,6.12,70B3D57050004D86
139764,2020-12-01 09:42:07.926000+00:00,2.50,70B3D57050001AB9
139763,2020-12-01 09:40:02.759000+00:00,4.18,70B3D5705000516A


In [82]:
# Creating an empty list and dataframe
latest_temps = pd.DataFrame()
devices = []

# For-looping the temps dataframe
for index, row in temps.iterrows():
    
    # Checking if device-id has been already stored in the list
    if row["dev-id"] not in devices:

        # Appending rows to the new dataframe
        latest_temps = latest_temps.append(row)
        
        # Adding the device-id to the list
        devices.append(row["dev-id"])
    
    # Since I know there are only 14 devices,
    # no reason to iterate over the whole dataset
    if len(devices) == 14:
        break


In [83]:
devices

['70B3D57050004FE6',
 '70B3D57050004C07',
 '70B3D57050001BBE',
 '70B3D57050004FE1',
 '70B3D57050001AB9',
 '70B3D57050005037',
 '70B3D57050004FB9',
 '70B3D57050004D86',
 '70B3D5705000516A',
 '70B3D57050004E0E',
 '70B3D57050001AF1',
 '70B3D57050004DF8',
 '70B3D57050001BA6',
 '70B3D5705000504F']

In [84]:
# Creating a string from the timestamp object
latest_temps["time"] = latest_temps.readable_time.dt.strftime('%Y-%m-%d %H:%M')


In [85]:
latest_temps

,dev-id,readable_time,temp_out1,time
139772,70B3D57050004FE6,2020-12-01 09:57:12.273000+00:00,4.56,2020-12-01 09:57
139771,70B3D57050004C07,2020-12-01 09:55:55.844000+00:00,4.50,2020-12-01 09:55
139770,70B3D57050001BBE,2020-12-01 09:55:54.960000+00:00,4.75,2020-12-01 09:55
139769,70B3D57050004FE1,2020-12-01 09:53:01.572000+00:00,5.25,2020-12-01 09:53
139768,70B3D57050001AB9,2020-12-01 09:52:09.101000+00:00,2.50,2020-12-01 09:52
139767,70B3D57050005037,2020-12-01 09:51:18.013000+00:00,6.25,2020-12-01 09:51
139766,70B3D57050004FB9,2020-12-01 09:45:55.084000+00:00,5.62,2020-12-01 09:45
139765,70B3D57050004D86,2020-12-01 09:44:58.703000+00:00,6.12,2020-12-01 09:44
139763,70B3D5705000516A,2020-12-01 09:40:02.759000+00:00,4.18,2020-12-01 09:40
139762,70B3D57050004E0E,2020-12-01 09:39:16.083000+00:00,4.12,2020-12-01 09:39


In [86]:
# Removing the unnecessary columns
latest_temps = latest_temps[["dev-id", "temp_out1", "time"]]

geo = geo[["name", "geometry"]]

In [87]:
geo

,name,geometry
70B3D57050001AB9,Pikkukosken uimaranta,POINT (2781186.232 8450609.098)
70B3D57050001BBE,Rastilan uimaranta,POINT (2795772.202 8446187.952)
70B3D57050004D86,Pihlajasaari,POINT (2773603.059 8431104.992)
70B3D57050004FB9,Hietaniemi (Ourit),POINT (2771666.078 8438019.854)
70B3D57050004C07,Sompasauna,POINT (2780239.682 8438969.276)
70B3D57050004DF8,Vasikkasaari,POINT (2784752.797 8433728.617)
70B3D57050004FE1,Herttoniemi (Tuorinniemen uimalaituri),POINT (2787453.408 8441162.681)
70B3D57050004FE6,Vartiosaari (Reposalmen laituri),POINT (2790623.787 8439944.788)
70B3D57050004E0E,Marjaniemen uimaranta,POINT (2791493.860 8444053.522)
70B3D5705000504F,Hanikan uimaranta (Espoo),POINT (2748686.506 8428245.610)


In [88]:
# Merging the datasets
join = geo.merge(latest_temps, left_index=True, right_on="dev-id")


In [89]:
join

,name,geometry,dev-id,temp_out1,time
139768,Pikkukosken uimaranta,POINT (2781186.232 8450609.098),70B3D57050001AB9,2.50,2020-12-01 09:52
139770,Rastilan uimaranta,POINT (2795772.202 8446187.952),70B3D57050001BBE,4.75,2020-12-01 09:55
139765,Pihlajasaari,POINT (2773603.059 8431104.992),70B3D57050004D86,6.12,2020-12-01 09:44
139766,Hietaniemi (Ourit),POINT (2771666.078 8438019.854),70B3D57050004FB9,5.62,2020-12-01 09:45
139771,Sompasauna,POINT (2780239.682 8438969.276),70B3D57050004C07,4.50,2020-12-01 09:55
139760,Vasikkasaari,POINT (2784752.797 8433728.617),70B3D57050004DF8,6.18,2020-12-01 09:34
139769,Herttoniemi (Tuorinniemen uimalaituri),POINT (2787453.408 8441162.681),70B3D57050004FE1,5.25,2020-12-01 09:53
139772,Vartiosaari (Reposalmen laituri),POINT (2790623.787 8439944.788),70B3D57050004FE6,4.56,2020-12-01 09:57
139762,Marjaniemen uimaranta,POINT (2791493.860 8444053.522),70B3D57050004E0E,4.12,2020-12-01 09:39
139732,Hanikan uimaranta (Espoo),POINT (2748686.506 8428245.610),70B3D5705000504F,4.93,2020-12-01 08:37


In [90]:
join.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [99]:
# Specifying the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specifying parameters 
params = dict(
    service='WFS',
    version='2.0.0',
    request='GetFeature',
    typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
    outputFormat='json'
)

# Fetching data from WFS using requests
r = requests.get(url, params=params)

# Creating GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Check the data
pop.head()

,geometry,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80
0,MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...,3342,108,45,11,23,6,7,26,17,8,6,4
1,MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...,3503,273,35,35,24,52,62,40,26,25,9,0
2,MULTIPOLYGON Z (((25476999.994 6675749.004 0.0...,3660,239,34,46,24,24,45,33,30,25,10,2
3,MULTIPOLYGON Z (((25476999.994 6675499.004 0.0...,3661,202,30,52,37,13,36,43,11,4,3,3
4,MULTIPOLYGON Z (((25476999.994 6675249.005 0.0...,3662,261,30,64,32,36,64,34,20,6,3,2


In [100]:
# Defining CRS
pop.crs = CRS.from_epsg(3879)

In [101]:
pop = pop.to_crs(epsg=4326)
join = join.to_crs(epsg=4326)

In [102]:
# Checking that CRS match
pop.crs == join.crs

True

In [103]:
# Changing the name of a column
pop = pop.rename(columns={"asukkaita": "pop18"})

In [104]:
# Creating a Geo-id which is needed by the Folium 
pop["geoid"] = pop.index.astype(str)

In [105]:
# Selecting only needed columns
pop = pop[['geoid', 'pop18', 'geometry']]

# Convert to geojson (not needed for the simple coropleth map!)
#pop_json = pop.to_json()

# Checking data
pop.head()

,geoid,pop18,geometry
0,0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2..."
1,1,273,"MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2..."
2,2,239,"MULTIPOLYGON Z (((24.58538 60.19391 0.00000, 2..."
3,3,202,"MULTIPOLYGON Z (((24.58541 60.19166 0.00000, 2..."
4,4,261,"MULTIPOLYGON Z (((24.58544 60.18942 0.00000, 2..."


In [149]:
# Reading data
fp = r"Helsingin_elintarvikehuoneistot.csv"
coffee = pd.read_csv(
    fp,
    encoding="unicode_escape",
    sep=';'
)
coffee

,Asiakasnro,Y-tunnus,Toimijan nimi1,Toimijan nimi2,TKohdenro,Kohde,Lahiosoite,Postitmp,Toimintaluokka,Toimintaluokan nimi,Toimintatyyppi,Toimintatyypin nimi,Asiakaspaikkojen määrä,Toiminnan aloitus pvm,Unnamed: 14
0,20803,1925435-0,Markkinointitoimisto Johdin Oy,NaN,35603,Markkinointitoimisto Johdin Oy,Liikkuva elintarvikehuoneisto,00100 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.20,Grilli- tai pikaruokatoiminta,-,NaN,NaN
1,20761,2860113-1,Saporita Panificazione Oy,NaN,35522,Piadineria Saporita,Unioninkatu 28,00100 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.20,Grilli- tai pikaruokatoiminta,4,NaN,NaN
2,20745,2138744-3,Findak,NaN,35486,Afrosene Food,Liikkuva elintarvikehuoneisto,00100 HELSINKI,E1008,ElintarvikkeidenTarjoilu,E1008.20,Grilli- tai pikaruokatoiminta,-,NaN,NaN
3,15177,2538302-8,Restel Fast Food Oy,NaN,35455,Burger King Oulunkylä,Isonpellontie 9,00720 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.20,Grilli- tai pikaruokatoiminta,136,NaN,NaN
4,20696,2892660-6,Daruma Oy,NaN,35383,Myyntivaunu/Daruma - Okonomiyaki,Liikkuva elintarvikehuoneisto,00100 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.20,Grilli- tai pikaruokatoiminta,-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,1140,0201256-6,Helsingin kaupunki kaupunkiympäristö,Rakennukset ja yleiset alueet/Tukkutori,11537,Töölöntori,Töölöntori,00260 HELSINKI,E1006,ElintarvikkeidenMyynti,E1006.90,Ulkomyyntialue,NaN,01.01.1900,NaN
4920,1140,0201256-6,Helsingin kaupunki kaupunkiympäristö,Rakennukset ja yleiset alueet/Tukkutori,11558,Ylä-Malmintori,Ylä-Malmintori,00700 HELSINKI,E1006,ElintarvikkeidenMyynti,E1006.90,Ulkomyyntialue,NaN,01.01.1900,NaN
4921,61,0201256-6,"Helsingin kaupunki kiinteistövirasto, tontti-o...",NaN,11532,Hietalahdentori,Hietalahdentori,00180 HELSINKI,E1006,ElintarvikkeidenMyynti,E1006.90,Ulkomyyntialue,NaN,01.01.1900,NaN
4922,3075,0712236-9,Kauppakeskus Itäkeskus Oy,NaN,18508,Tallinnanaukion tori,Tallinnanaukio,00930 HELSINKI,E1006,ElintarvikkeidenMyynti,E1006.90,Ulkomyyntialue,NaN,09.01.2012,NaN


In [154]:
# Selecting only the cafés 
cafe = coffee.loc[coffee["Toimintatyypin nimi"] == "Kahvilatoiminta"]
cafe

,Asiakasnro,Y-tunnus,Toimijan nimi1,Toimijan nimi2,TKohdenro,Kohde,Lahiosoite,Postitmp,Toimintaluokka,Toimintaluokan nimi,Toimintatyyppi,Toimintatyypin nimi,Asiakaspaikkojen määrä,Toiminnan aloitus pvm,Unnamed: 14
266,20823,2913734-2,Fysioterapia Jenni Kuusela,NaN,35645,Cafe Kulo,Soutajankuja 2,00570 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,10,NaN,NaN
267,7108,0986855-1,Kanresta Oy,NaN,35640,Kansallismuseon kahvila,Mannerheimintie 34,00100 HELSINKI,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,70,NaN,NaN
268,20814,2777894-7,HKP Päiväkahvibaari Oy,NaN,35626,Päiväkahvibaari,Päijänteentie 29,00510 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,6,NaN,NaN
269,20795,2393898-2,Helsingin Tili ja Tase Oy,NaN,35605,Cafe MIGO,Bulevardi 15,00120 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,6,NaN,NaN
270,20804,2661404-3,Saura Ohjelmapalvelu Oy,NaN,35601,Saurahuone,Pohjolankatu 43,00610 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,28,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,21756,2951813-7,Bar Milano Helsinki Oy,NaN,37681,La Lanterna,Viides linja 7,00530 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,30,NaN,NaN
966,21763,2974858-1,Viapori Spirit Oy,NaN,37694,Linna,Suomenlinna C 70,00190 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,50,NaN,NaN
967,21206,2923485-5,Voima Kahvila Oy,NaN,37719,Voimakahvila,Teollisuuskatu 27,00510 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,18,NaN,NaN
968,21705,2939696-9,Wajadee Coffeehouse Oy,NaN,37721,Cafe Papu,Kap Horninkatu 3 LT 1,00210 Helsinki,E1008,ElintarvikkeidenTarjoilu,E1008.30,Kahvilatoiminta,50,NaN,NaN


In [157]:
from geopandas.tools import geocode

# Geocoding the addresses
geo = geocode(cafe['Lahiosoite'], provider='nominatim', user_agent='autogis_xx', timeout=10)

GeocoderServiceError: Non-successful status code 500

In [110]:
import folium

my_tooltip = folium.features.GeoJsonTooltip(
    fields=["name", "time", "temp_out1"],
    aliases=["Place:", "Time of measurement", "Temperature (°C)"],
    labels=True,
    sticky=False
)

# This popup doesnt work, maybe due to a bug
#my_popup = folium.features.GeoJsonPopup(
#    fields=["name", "time", "temp_out1"],
#    aliases=["Place:", "Measured at:", "Temperature (°C):"],
#    localize=True,
#    parse_html=True
#)

point_gjson = folium.features.GeoJson(
    join,
    name="Beaches",
    popup=my_popup,
    tooltip=my_tooltip
)

m = folium.Map(
    location=[60.25, 24.95],
    zoom_start=11,
    control_scale=True,
    tiles="cartodbpositron"
)


point_gjson.add_to(m)

# Plot a choropleth map
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=pop,
    data=pop,
    columns=["geoid", "pop18"],
    key_on="feature.id",
    fill_color="YlOrRd",
    line_weight=0
).add_to(m)

#folium.TileLayer('Mapbox Bright').add_to(m)

#folium.GeoJson(join,
#    tooltip=my_tooltip,
#    popup=my_popup).add_to(m)

#markers = folium.Marker(point_gjson)

folium.LayerControl().add_to(m)

#markers = folium.Marker(popup="Uimaranta", tooltip="kalupiste")

#markers.add_to(m)
    
m   
    